This notebook evaluates the performance of adversarial patches (i.e. noise, black, checker, and universal (trained)).

# Setup
Imports, Installations, and Downloads

In [ ]:
import shutil; shutil.rmtree("/root/fiftyone/open-images-v7/", ignore_errors=True)

In [ ]:
!pip install -q transformers timm torchvision matplotlib

In [ ]:
!pip install -q tfds-nightly tensorflow matplotlib

In [ ]:
import torch
import torchvision.transforms as T
import matplotlib.pyplot as plt
from PIL import Image
import requests
import numpy as np
import tensorflow_datasets as tfds
from transformers import DetrImageProcessor, DetrForObjectDetection

processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")
model.eval()


In [ ]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
!pip install -U fiftyone

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz

In [ ]:
dataset = foz.load_zoo_dataset(
    "open-images-v7",
    split="validation",
    max_samples=100,
    seed=51,
    shuffle=True,
)

extra = foz.load_zoo_dataset(
    "open-images-v7",
    split="test",
    max_samples=400,
    seed=70,
    shuffle=True
)
dataset.add_samples(extra)


In [ ]:
# Get a sample from dataset
sample = dataset.first()
image = Image.open(sample.filepath).convert("RGB")

In [ ]:
import torchvision.ops as ops
import matplotlib.patches as patches
import csv

# Create Patches & Define Helper Functions

In [ ]:
# Sum of the distances the bounding boxes moved
def detections_distances(clean_outputs, patched_outputs, threshold=0.0):
    clean_probs = clean_outputs.logits[0].softmax(-1)
    patched_probs = patched_outputs.logits[0].softmax(-1)

    clean_keep = clean_probs.max(-1).values > threshold
    patched_keep = patched_probs.max(-1).values > threshold

    clean_boxes = clean_outputs.pred_boxes[0][clean_keep][:, :2]  # cx, cy
    patched_boxes = patched_outputs.pred_boxes[0][patched_keep][:, :2]

    distances = []
    if clean_boxes.shape[0] != 0 and patched_boxes.shape[0] != 0:
      for i, clean_center in enumerate(clean_boxes):
          dists = torch.norm(patched_boxes - clean_center, dim=1)
          min_dist = dists.min().item()
          distances.append(min_dist)
    return sum(distances)


In [ ]:
transform = T.Compose([
    T.Resize((480, 640)),
    T.ToTensor()
])

# Helper functions for comparing adversarial/clean results
def apply_patch(img, patch, x=300, y=200):
    img_clone = img.clone()
    img_clone[:, :, y:y+patch_size, x:x+patch_size] = patch
    return img_clone

def count_real_detections(outputs, threshold=0.5, no_object_class=91):
    probs = outputs.logits[0].softmax(-1)
    scores, labels = probs.max(-1)
    return ((scores > threshold) & (labels != no_object_class)).sum().item()

def get_boxes(outputs, conf_thresh=0.9, no_object_class=91):
    probs = outputs.logits[0].softmax(-1)
    scores, labels = probs.max(-1)
    keep = (scores > conf_thresh) & (labels != no_object_class)
    boxes = outputs.pred_boxes[0][keep]
    H, W = 480, 640
    boxes_xyxy = []
    for box in boxes:
        cx, cy, w, h = box
        x0 = (cx - w/2) * W
        y0 = (cy - h/2) * H
        x1 = (cx + w/2) * W
        y1 = (cy + h/2) * H
        boxes_xyxy.append([x0.item(), y0.item(), x1.item(), y1.item()])
    return torch.tensor(boxes_xyxy)

def box_suppression_ratio(clean_boxes, patched_boxes, iou_thresh=0.5):
    if len(clean_boxes) == 0:
        return 0.0
    iou_matrix = ops.box_iou(clean_boxes, patched_boxes)
    max_iou_per_clean = iou_matrix.max(dim=1).values
    unmatched = (max_iou_per_clean < iou_thresh).sum().item()
    return unmatched / len(clean_boxes)

def detection_entropy(outputs):
    probs = outputs.logits[0].softmax(-1)
    entropies = -torch.sum(probs * probs.log(), dim=-1)
    return entropies.mean().item()

def show_detections(img_tensor, outputs, title):
    img = img_tensor.squeeze().permute(1, 2, 0).detach().cpu().numpy()

    boxes = outputs.pred_boxes[0].detach().cpu()
    probs = outputs.logits[0].softmax(-1).detach().cpu()
    keep = probs.max(-1).values > 0.9

    plt.figure(figsize=(8, 8))
    plt.imshow(img)
    H, W = img_tensor.shape[-2:]

    for box in boxes[keep]:
        cx, cy, w, h = box
        x0 = (cx - w/2) * W
        y0 = (cy - h/2) * H
        rect = patches.Rectangle((x0, y0), w*W, h*H, edgecolor='red', facecolor='none', linewidth=2)
        plt.gca().add_patch(rect)

    plt.title(title)
    plt.axis("off")
    plt.show()


In [ ]:
patch_size = 80
tile_size = 10

# Baseline patches
black_patch = torch.zeros((1, 3, patch_size, patch_size)).to(device)
checkered_patch = (((torch.arange(patch_size).unsqueeze(1) // tile_size +
              torch.arange(patch_size).unsqueeze(0) // tile_size) % 2)
            .float().unsqueeze(0).repeat(3,1,1).unsqueeze(0)).to(device)
noisy_patch = torch.rand((1, 3, patch_size, patch_size)).to(device)

# Trained patch
optim_patch = checkered_patch.clone().detach() # checkered start; seems to work best
optim_patch = optim_patch.to(device)
optim_patch.requires_grad = True
optimizer = torch.optim.Adam([optim_patch], lr=0.8e-2)

In [ ]:
import torch.nn.functional as F

# Calculate the loss to maximize shifts in bounding boxes & differences in predictions
def combined_loss(clean_outputs, patched_outputs):
    shift_loss = -detections_distances(clean_outputs, patched_outputs)

    clean_probs = clean_outputs.logits.softmax(dim=-1)
    patched_log_probs = F.log_softmax(patched_outputs.logits, dim=-1)
    class_diff_loss = -F.kl_div(patched_log_probs, clean_probs, reduction='batchmean')

    alpha = 0.5
    beta = 1
    total_loss = alpha * class_diff_loss + beta * shift_loss
    return total_loss

# Train the patch
for epoch in range(10):
    model.train()
    total_train_loss = 0
    for sample in dataset[100:]: # 400 samples total
        image = Image.open(sample.filepath).convert("RGB")
        img_tensor = transform(image).unsqueeze(0).to(device)

        optimizer.zero_grad()
        patched_img = apply_patch(img_tensor, optim_patch).to(device)
        clean_outputs = model(img_tensor)
        patched_outputs = model(patched_img)
        loss = combined_loss(clean_outputs, patched_outputs)
        loss.backward()
        optimizer.step()
        optim_patch.data.clamp_(0, 1)

        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(dataset[100:])

    # Validation
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for sample in dataset[25:100]: # 75 samples
            image = Image.open(sample.filepath).convert("RGB")
            img_tensor = transform(image).unsqueeze(0).to(device)

            patched_img = apply_patch(img_tensor, optim_patch).to(device)
            clean_outputs = model(img_tensor)
            patched_outputs = model(patched_img)
            val_loss = combined_loss(clean_outputs, patched_outputs)
            total_val_loss += val_loss.item()

    avg_val_loss = total_val_loss / len(dataset[25:100])

    print(f"Epoch {epoch + 1} completed, Train Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")


Epoch 1 completed, Train Loss: -52.8710, Validation Loss: -108.2958
Epoch 2 completed, Train Loss: -115.0496, Validation Loss: -138.8757
Epoch 3 completed, Train Loss: -129.1578, Validation Loss: -150.9644
Epoch 4 completed, Train Loss: -134.8828, Validation Loss: -156.1643
Epoch 5 completed, Train Loss: -139.9937, Validation Loss: -162.6386
Epoch 6 completed, Train Loss: -147.1126, Validation Loss: -164.2050
Epoch 7 completed, Train Loss: -152.4851, Validation Loss: -172.6912
Epoch 8 completed, Train Loss: -155.7269, Validation Loss: -173.9079
Epoch 9 completed, Train Loss: -157.3030, Validation Loss: -174.9588
Epoch 10 completed, Train Loss: -158.4956, Validation Loss: -181.5939


In [ ]:
# Save patch
final_patch = optim_patch
torch.save(final_patch, "final_patch.pt")

# Analyze Patches
Analyze patch on test set (+ visualize bounding boxes)

In [ ]:
# Print/save analysis of the effects of the patch
def analyze_patch(patch, patch_name):
    results = []
    print(patch_name)

    for sample in dataset[:25]:
        image = Image.open(sample.filepath).convert("RGB")
        img_tensor = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            proc_clean = processor(images=img_tensor, return_tensors="pt")
            proc_clean = {k: v.to(device) for k, v in proc_clean.items()}
            clean_outputs = model(**proc_clean)

            patched_tensor = apply_patch(img_tensor, patch).to(device)
            proc_patched = processor(images=patched_tensor, return_tensors="pt")
            proc_patched = {k: v.to(device) for k, v in proc_patched.items()}
            patched_outputs = model(**proc_patched)

        clean_boxes = get_boxes(clean_outputs)
        patched_boxes = get_boxes(patched_outputs)

        shift = detections_distances(clean_outputs, patched_outputs)
        suppression_ratio = box_suppression_ratio(clean_boxes, patched_boxes)
        entropy_clean = detection_entropy(clean_outputs)
        entropy_patched = detection_entropy(patched_outputs)

        # Compare predicted classes by argmax on logits
        clean_logits = clean_outputs.logits
        patched_logits = patched_outputs.logits

        clean_preds = clean_logits.argmax(dim=-1).squeeze(0)
        patched_preds = patched_logits.argmax(dim=-1).squeeze(0)

        # Calculate how many predictions match
        matches = (clean_preds == patched_preds).sum().item()
        total_preds = clean_preds.shape[0]
        match_ratio = matches / total_preds if total_preds > 0 else 0.0

        print(f"{sample.filepath} — suppression: {suppression_ratio:.2f}, shift: {shift:.2f}, entropy: {entropy_clean:.2f} → {entropy_patched:.2f}, match_ratio: {match_ratio:.2f}")

        results.append({
            "filename": sample.filepath,
            "suppression_ratio": suppression_ratio,
            "sum_box_shift": shift,
            "entropy_clean": entropy_clean,
            "entropy_patched": entropy_patched,
            "prediction_match_ratio": match_ratio
        })

        show_detections(img_tensor, clean_outputs, "Original")
        show_detections(patched_tensor, patched_outputs, "After Patch")

    csv_path = "/content/results_" + patch_name + ".csv"
    with open(csv_path, "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=[
            "filename",
            "suppression_ratio", "sum_box_shift",
            "entropy_clean", "entropy_patched", "prediction_match_ratio"
        ])
        writer.writeheader()
        for row in results:
            writer.writerow(row)


In [ ]:
print(optim_patch.device)
model = model.to(device)
model.eval()
analyze_patch(optim_patch, "Optimized Patch")

In [ ]:
analyze_patch(noisy_patch, "Noisy Patch")

In [ ]:
analyze_patch(black_patch, "Black Patch")

In [ ]:
analyze_patch(checkered_patch, "Checkered Patch")